# Notebook on hi ha la versió millorada del recomanador. Hi ha el codi que ens ha generat la puntuació més alta al Kaggle

# Sistemes de Recomanació

En aquest segon lliurament es programarà un **sistema de recomanació**, que posarà en correspondència un *usuari* amb *ítems* en funció de les seves preferències i interessos. 
En aquesta ocasió, implementareu un sistema de recomanació que assisteixi en una compra de supermercat.

## Abans de començar


**\+ Durant la pràctica, solament es podran fer servir les següents llibreries**:

`Pandas, Numpy, Itertools`

*Nota: A més de les que ja es troben presents en la 1a cel·la i funcions natives de Python*

**\+ No es poden modificar les definicions de les funcions donades, ni canviar els noms de les variables i paràmetres ja donats**

Això no implica però que els hàgiu de fer servir. És a dir, que la funció tingui un paràmetre anomenat `df` no implica que l'hàgiu de fer servir, si no ho trobeu convenient.

**\+ En les funcions, s'especifica que serà i de quin tipus cada un dels paràmetres, cal respectar-ho**

Per exemple (ho posarà en el pydoc de la funció), `df` sempre serà indicatiu del `Pandas.DataFrame` de les dades. Durant els testos, els paràmetres (i específicament `df`) no contindran les mateixes dades que en aquest notebook, si bé si seran del mateix tipus! Per tant, no us refieu de què tinguin, per exemple, el mateix nombre de files.

# Preparar les dades

### **En aquesta cel·la no féu cap modificació**

Descomprimeix els zips a la carpeta "data" automàticament. 

Descarregueu el zip amb les dades del campus i guardeu-lo dins de la carpeta del projecte. **No pugeu cap fitxer de dades a Github** (ja incloem al .gitignore un patró per ignorar-los).

In [1]:
import zipfile
import pickle
from os.path import join, dirname

def unzip(file):
    zip_ref = zipfile.ZipFile(file, 'r')
    zip_ref.extractall('data')
    zip_ref.close()
    
unzip('dades_p1.zip')

# Les dades

En aquest i futur notebooks farem servir dades reals corresponents a compres, concretament les utilitzades en el Kaggle Instacart Market Basket Analysis:
https://www.kaggle.com/c/instacart-market-basket-analysis


* **Order Products**: És el de major interès, conté la relació de productes comprats (`product_id`) per a cada conjunt de compra diferent (`order_id`). A aquests conjunts de compres ens hi referirem com a `ordres`, seguint la nomenclatura de les dades. A més, tot i que no ho farem servir, podríem arribar a saber en quin ordre s'han comprat els productes (`add_to_cart_order`) i inclús si ja s'havia comprat en alguna ordre anterior (`reordered`).

* **Orders**: Aquest dataset ens permet relacionar una compra en concret (`order_id`) amb l'usuari que l'ha feta (`user_id`)

* **Products**: Donat un `product_id` ens permet obtenir-ne més informació, com ara el nom (`product_name`), la secció en la qual es troba (`aisle_id`) o al departament al qual pertany (`department_id`). Aquests dos últims es complementen amb els conjunts **Aisles** i **Departments**.


# Data loading

### **En aquestes cel·les no feu cap modificació**

Carrega les dades en un DataFrame Pandas.

In [2]:
import pandas as pd

if __name__ == '__main__':
    df_order_prods = pd.read_csv(join('data', 'order_products__train.csv'))
    df_orders = pd.read_csv(join('data', 'orders.csv'))[['order_id', 'user_id']]
    df_prods = pd.read_csv(join('data', 'products.csv'))[['product_id', 'aisle_id']]

# Implementació

Recordeu, seguiu els pydoc i compliu amb el que diuen!

El primer que haurem de fer és construir una matriu que ens serveixi, d'alguna forma, com a indicatiu de preferències de cada persona. Per tal efecte, construirem una matriu $m\times n$, de $m$ usuaris per $n$ items, on cada entrada $i,j$ serà el nombre de vegades que la persona $i$ a comprat l'item $j$.

<img src="img/Mat.png">

Per saber de quin usuari és cada `order_id`, haureu de creuar el dataset `order_products` amb el `orders`. Una sola persona/usuari tindrà més d'una ordre, mireu quants cops ha comprat els mateixos productes.

A més, les dades es componen de molts `product_id diferents`, hi ha massa diversitat entre usuaris. Per tant, per poder recomanar el que farem serà agregar les dades, en lloc de treballar per `product_id` ho farem per `aisle_id`, és a dir "la secció" del súper on es troba.

Al llarg de la pràctica es parlarà de producte i/o item, perquè és la terminologia estàndard de recomanadors, però sempre serà en referència a `aisle_id` per aquesta pràctica!

In [3]:
def merge_information(df_order_prods, df_orders, df_prods):
    """
    Retorna el dataframe resultant de:
        1. Creuar els datasets 'order_products' amb 'orders'.
        2. Creuar el dataframe anterior amb 'products'.
        Per creuar dos dataframes podeu utilitzar la funció pandas.DataFrame.merge

    :param df_order_prods: DataFrame 'order_products'
    :param df_orders: DataFrame 'orders'
    :param df_prods: DataFrame 'products'
    :return: DataFrame descrit prèviament   
    """
    return pd.merge(pd.merge(df_order_prods, df_orders), df_prods)

In [4]:
if __name__ == '__main__':
    df_merged = merge_information(df_order_prods, df_orders, df_prods)

In [5]:
def build_counts_table(df):
    """
    Retorna un dataframe on les columnes són els `aisle_id`, les files `user_id` i els valors
    el nombre de vegades que un usuari ha comprat un producte d'un `aisle_id`
    
    :param df: DataFrame original després de creuar-lo
    :return: DataFrame descrit adalt
    """
    return pd.crosstab(df['user_id'], df['aisle_id'])

def get_count(df, user_id, aisle_id):
    """
    Retorna el nombre de vegades que un usuari ha comprat en un `aisle_id`
    
    :param df: DataFrame retornat per `build_counts_table`
    :param user_id: ID de l'usuari
    :param aisle_id: ID de la secció
    :return: Enter amb el nombre de vegades que ha comprat
    """
    return df.loc[user_id, aisle_id]

In [6]:
if __name__ == '__main__':
    df_counts = build_counts_table(df_merged)
    count = get_count(df_counts, 14, 5)
    print(count)

2


Tenim moltes dades en el nostre dataset, pel que és convenient que les reduïm una mica. Per començar a treballar recomanem que reduïu la mida a aproximadament 0.1 de l'original '(FRAC = 0.1)'.

In [7]:
# Funcio creada per intentar millorar la puntuacio del Kaggle (finalment, pel resultat mes alt que hem obtingut no l'hem
# utilitzada)
def filter_data(df_counts):
    """
    Retorna el dataframe resultant d'aplicar una neteja a les dades del parametre
    df_counts. Aquesta neteja consisteix en treure els usuaris que han comprat molt
    uniformement tots els productes i els usuaris que han comprat molt pocs productes.
    
    :param df: DataFrame a filtrar
    :return: Dataframe resultant d'aplicar una neteja a les dades de 'df_counts'
    """
    # Eliminem els usuaris que hagin comprat molt uniformement, es a dir, aquells en que el vector format pel nombre d'items
    # que han comprat de cada tipus de 'aisle_id' tingui una desviacio estandard petita.
    users_filtered = df_counts[df_counts.std(axis=1) > 0.15]
    
    # Eliminem els usuaris que hagin pocs items (hem decidit eliminar els que hagin comprat menys items que la meitat de
    # la mitjana d'items que ha comprat cada usuari)
    mean = users_filtered.sum(axis=1).mean()
    users_filtered = users_filtered[users_filtered.sum(axis=1) > 0.5*mean]
    
    return users_filtered

In [8]:
if __name__ == '__main__':
    FRAC = 0.0055
    df_reduced = df_counts.sample(frac=FRAC, random_state=1)

### Mesurament de similituds

El primer pas per poder recomanar és definir una funció de similitud entre vectors. Siguin $x, y$ vectors, de les següents propostes implementa'n mínim una:

* Distància euclidea (inversa): https://en.wikipedia.org/wiki/Euclidean_distance

$$sim(x, y) = \frac{1}{1 + \sqrt{\sum_i(x_i-y_i)^2}}\in [0, 1]$$

* Similitud cosinus: https://en.wikipedia.org/wiki/Cosine_similarity

$$sim(x, y) = \frac{x\cdot y}{||x||\hspace{0.1cm} ||y||} \in [-1,1]$$

* Correlació de Pearson: https://en.wikipedia.org/wiki/Pearson_correlation_coefficient

$${\displaystyle sim(x,y)={\frac {\sum _{i=1}^{n}(x_{i}-{\bar {x}})(y_{i}-{\bar {y}})}{{\sqrt {\sum _{i=1}^{n}(x_{i}-{\bar {x}})^{2}}}{\sqrt {\sum _{i=1}^{n}(y_{i}-{\bar {y}})^{2}}}}}} \in [-1,1] \\ \text{On }\bar{x} = \frac{1}{n} \sum^n_i x_i\text{ la mitja (i anàlogament per y)}$$

Per implementar qualsevol d'aquestes únicament es permet l'ús de:

* `np.sum`
* `np.sqrt`
* `np.power`
* `np.dot`
* `np.linalg.norm`
* `np.mean`

I s'ha de fer **sense bucles**.

<hr>

Tingueu en compte que les dues últimes funcions consideren valors negatius per exemples oposats (a diferència de la distància euclidea). En cas de fer servir alguna d'aquestes dues, pensa (més endavant) en com afectaran els negatius en la recomanació.

En la similitud cosinus, vigileu amb casos on un usuari no ha comprat res, tindreu a ser una divisió entre 0.

En la correlació de Pearson, haureu de considerar casos on algun dels dos exemples tingui variància 0, ja que aleshores estareu fent una divisió entre 0. En tals casos, podeu retornar un valor per defecte o alguna de les altres mesures.

In [9]:
import numpy as np

def similarity(x, y):
    """
    Definir quina de les similituds vols utilitzar  a l'execució.
    
    :param x: Primer vector
    :param y: Segon vector
    :return : Escalar (float) corresponent a la similitud
    """
    return pearson(x, y)
    
def euclid(x, y):
    """
    Retorna la distància euclidiana inversa de dos vectors n-dimensionals.
    
    :param x: Primer vector
    :param y: Segon vector
    :return : Escalar (float) corresponent a la distància euclidiana
    """
    return 1/(1+np.linalg.norm(x-y, ord=2))

def cosine(x, y):
    """
    Retorna la similitud cosinus de dos vectors n-dimensionals.
    
    :param x: Primer vector
    :param y: Segon vector
    :return : Escalar (float) corresponent a la similitud cosinus
    """
    x_norm = np.linalg.norm(x, ord=2)
    y_norm = np.linalg.norm(y, ord=2)
    
    # Si alguna de les dues normes es 0, significa que totes les components d'un dels dos vectors son 0, es a dir,
    # que no tenim informacio d'un dels dos vectors. Aixi, retornem -1 com a resultat perque no estan gens relacionats.
    if (x_norm == 0 or y_norm == 0):
        return -1
    #Altrament retornem la formula de la similitud cosinus
    return np.dot(x, y)/(x_norm * y_norm)

def pearson(x, y):
    """
    Retorna la correlació de Pearson de dos vectors n-dimensionals.
    
    :param x: Primer vector
    :param y: Segon vector
    :return : Escalar (float) corresponent a la correlació de Pearson
    """
    first = x - np.mean(x)
    second = y - np.mean(y)
    norm_first = np.linalg.norm(first, ord=2)
    norm_second = np.linalg.norm(second, ord=2)
    
    # Si alguna de les dues normes es 0, significa que un dels dos vectors te totes les components iguals i a la
    # formula estariem dividint per 0. En aquest cas, hem decidit retornar la similitud cosinus entre els dos vectors
    # perque la similitud cosinus distingira entre si totes les components del vector son 0 o iguals pero diferents de 0.
    if ((norm_first == 0) or (norm_second == 0)):
        return cosine(x, y)
    #Altrament retornem la formula de la correlacio de Pearson
    return np.dot(first, second)/(norm_first*norm_second)

In [10]:
if __name__ == '__main__':
    print(similarity(np.asarray([1, 1, 1]), np.asarray([1, 2, 3])))

0.9258200997725515


### Matriu de similituds

Per fer recomanació col·laborativa existeixen dues opcions, fer un recomanador basat en usuaris o un en ítems:

* Recomanador basat en usuaris:
Considera la matriu $M\times N: \text{usuaris}\times\text{items}$, per recomanar t'hauràs de basar en les similituds entre els usuaris.

* Recomanador basat en items:
Considera la matriu $M\times N: \text{items}\times\text{usuaris}$, per recomanar t'hauràs de basar en les similituds entre els ítems.

Construeix una matriu de mida $M\times M$ on cada posició $i,j$ indica la distància entre l'element $i$ i el $j$. Així doncs, si estàs fent un recomanador basat en usuaris, `matriu[2, 3]` contindrà la similitud entre l'usuari 2 i el 3. En canvi, si l'estàs fent basat en ítems, `matriu[2, 3]` contindrà la similitud entre l'ítem 2 i el 3.

In [26]:
# Hem decidit implementar la funcio similarity_matrix tal i com diu l'apartat opcional de la cel.la seguent.
# Es a dir, que el primer parametre es una funcio que hem definit nosaltres que treballa especificament amb matrius
# (i no vectors). Hem decidit fer-ho aixi perque tot i que sembla que hi ha molt mes codi, representa una millora
# molt important a nivell temporal, sobretot per a valors grans de FRAC.

# Funcio que usarem com a parametre per a la funcio similarity_matrix
def iter_matrix_similarity(counts, roll_counts):
    """
    Retorna un vector numpy on a la component i-essima hi ha el calcul de la similitud entre la
    fila i-essima de la matriu 'counts' i la fila i-essima de la matriu 'roll_counts'.
    
    :param counts: matriu de mida M x N
    :param roll_counts: matriu de mida M x N
    :return: Vector numpy de mida M on a la component i-essima hi ha el calcul de la similitud entre la
    fila i-essima de la matriu 'counts' i la fila i-essima de la matriu 'roll_counts'.
    """
    # Hem decidit usar la correlacio de Pearson com a funcio de similitud, pero podriem usar qualsevol de les 3 funcions
    # definides a continuacio
    return iter_matrix_pearson(counts, roll_counts)
    

# Funcio creada per nosaltres per poder fer el calcul de la matriu de similituds de forma matricial
# usant la distancia euclidiana (inversa).
def iter_matrix_euclid(counts, roll_counts):
    """
    Retorna un vector numpy on a la component i-essima hi ha el calcul de la distancia euclidiana (inversa) entre la
    fila i-essima de la matriu 'counts' i la fila i-essima de la matriu 'roll_counts'.
    
    :param counts: matriu de mida M x N
    :param roll_counts: matriu de mida M x N
    :return: Vector numpy de mida M on a la component i-essima hi ha el calcul de la distancia euclidiana (inversa) entre la
    fila i-essima de la matriu 'counts' i la fila i-essima de la matriu 'roll_counts'.
    """
    # Restem les dues matrius i calculem les normes euclidianes per files
    partial_sol = np.linalg.norm(counts - roll_counts, ord=2, axis=1)
    # Retornem el vector amb la distancia euclidiana (inversa) entre les files de les dues matrius 
    return 1 / (1 + partial_sol)

# Funcio creada per nosaltres per poder fer el calcul de la matriu de similituds de forma matricial
# usant la similitud cosinus.
def iter_matrix_cosine(counts, roll_counts):
    """
    Retorna un vector numpy on a la component i-essima hi ha el calcul de la similitud cosinus entre la
    fila i-essima de la matriu 'counts' i la fila i-essima de la matriu 'roll_counts'.
    
    :param counts: matriu de mida M x N
    :param roll_counts: matriu de mida M x N
    :return: Vector numpy de mida M on a la component i-essima hi ha el calcul de la similitud cosinus entre la
    fila i-essima de la matriu 'counts' i la fila i-essima de la matriu 'roll_counts'.
    """
    # Calculem la norma euclidiana per a cada fila de la matriu counts
    counts_norm = np.linalg.norm(counts, ord=2, axis=1)
    # Calculem la norma euclidana per a cada fila de la matriu roll_counts
    roll_counts_norm = np.linalg.norm(roll_counts, ord=2, axis=1)
    
    # Si la norma euclidiana de totes les files de la matriu counts i de totes les files de la matriu roll_counts son diferents
    # de 0, podem aplicar la similitud cosinus entre cada fila de la matriu counts i de la matriu roll_counts
    if (np.all(counts_norm != 0) and np.all(roll_counts_norm != 0)):
        # Retornem el vector amb la similitud cosinus entre cada fila de les dues matrius
        return np.sum(counts*roll_counts, axis=1)/(counts_norm*roll_counts_norm)
    
    # Si arribem en aquest punt, significa que en el calcul d'alguna similitud cosinus estariem dividint per 0.
    # Creem un vector de zeros que omplirem amb la similitud entre cada fila de les dues matrius
    result = np.zeros(counts.shape[0], dtype='float64')
    
    # Creem una mascara per saber a quines files de la matriu no estariem dividint per 0 si calculessim la similitud cosinus
    mask = np.logical_and(counts_norm != 0, roll_counts_norm != 0)
    # En aquelles files on es pugui calcular la similitud cosinus, la calculem
    result[mask] = np.sum(counts[mask]*roll_counts[mask], axis=1) / (counts_norm[mask]*roll_counts_norm[mask])
    # En aquelles files on no es pugui calcular la similitud cosinus perque estariem dividint per 0, posem la similitud a -1,
    # seguint la mateixa logica explicada a la funcio cosine de la cel.la anterior
    result[np.logical_not(mask)] = -1
    # Retornem el vector amb la similitud cosinus entre cada fila de les dues matrius 
    return result

# Funcio creada per nosaltres per poder fer el calcul de la matriu de similituds de forma matricial
# usant la correlacio de Pearson.
def iter_matrix_pearson(counts, roll_counts):
    """
    Retorna un vector numpy on a la component i-essima hi ha el calcul de la correlacio de Pearson entre la
    fila i-essima de la matriu 'counts' i la fila i-essima de la matriu 'roll_counts'.
    
    :param counts: matriu de mida M x N
    :param roll_counts: matriu de mida M x N
    :return: Vector numpy de mida M on a la component i-essima hi ha el calcul de la correlacio de Pearson entre la
    fila i-essima de la matriu 'counts' i la fila i-essima de la matriu 'roll_counts'.
    """
    #Restem a cada element de les matrius la mitjana de tota la fila de la matriu
    first = counts - counts.mean(axis=1, keepdims=True)
    second = roll_counts - roll_counts.mean(axis=1, keepdims=True)
    
    #Calculem les normes euclidianes de les files de cadascuna de les dues matrius
    first_norm = np.linalg.norm(first, ord=2, axis=1)
    second_norm = np.linalg.norm(second, ord=2, axis=1)
    
    # Si les normes euclidianes de totes les files de la matriu first i de totes les files de la matriu second son diferents
    # de 0, podem aplicar la correlacio de Pearson entre cada fila de la matriu counts i de la matriu roll_counts
    if (np.all(first_norm != 0) and np.all(second_norm != 0)):
        # Retornem el vector amb la correlacio de Pearson entre cada fila de les dues matrius
        return np.sum(first*second, axis=1)/(first_norm*second_norm)
    
    # Si arribem en aquest punt, significa que en el calcul d'alguna correlacio de Pearson estariem dividint per 0.
    # Creem un vector de zeros que omplirem amb la similitud entre cada fila de les dues matrius
    result = np.zeros(first.shape[0], dtype='float64')
    # Creem una mascara per saber a quines files de la matriu no estariem dividint per 0 si calculessim la correlacio de Pearson
    mask = np.logical_and(first_norm != 0, second_norm != 0)
    # En aquelles files on es pugui calcular la correlacio de Pearson, la calculem
    result[mask] = np.sum(first[mask]*second[mask], axis=1)/(first_norm[mask]*second_norm[mask])
    # Invertim la mascara per saber entre quines files no es pot calcular la correlacio de Pearson
    inverted_mask = np.logical_not(mask)
    # En aquelles files on no es pugui calcular la correlacio de Pearson perque estariem dividint per 0,
    # calculem la similitud cosinus, seguint la mateixa logica explicada a la funcio pearson de la cel.la anterior
    result[inverted_mask] = iter_matrix_cosine(counts[inverted_mask], roll_counts[inverted_mask])
    # Retornem el vector amb la correlacio de Pearson entre cada fila de les dues matrius
    return result

def similarity_matrix(similarity_function, df_counts):
    """
    Retorna una matriu de mida M x M on cada posició 
    indica la similitud entre usuaris (resp. ítems).
    
    :param similarity_function: Funció que calcularà la similitud 
        entre usuaris (resp. ítems)
    :param df_counts: Dataframe que conté el nombre de vegades que 
        un usuari ha comprat en un `aisle_id`
    :return : Matriu numpy de mida M x M amb les similituds.
    """
    
    num_users = df_counts.shape[0]
    # Convertim el Dataframe a numpy per fer mes rapid els calculs i poder fer el calcul de la similituds de forma matricial 
    np_counts = df_counts.values
    # Creem una matriu de zeros que anirem omplint amb les similituds a mesura que les anem calculant
    sim_matrix = np.zeros((num_users, num_users), dtype='float64')
    
    # Guardem una copia de la matriu counts. Aquesta copia ens servira per anar-la rotant i anar
    # calculant la similitud entre totes les files de la matriu counts
    np_roll_counts = np_counts.copy()
    
    # El nombre d'iteracions que necessitarem usant el metode matricial es (num_users+2)/2 si hi ha un nombre parell
    # d'usuaris i (n+1)/2 si hi ha un nombre senar d'usuaris. Aixo es perque la matriu de similituds es simetrica.
    num_iter = int((num_users+2) / 2)
    if (num_users % 2 != 0):
        num_iter = int((num_users+1) / 2)
    # Calculem el nombre de columnes (nombre d'items) que te el Dataframe original
    num_cols = np_roll_counts.shape[1]

    for i in range(num_iter):
        
        # Creem un vector numpy anomenat partial_sol on a cada component i-essima hi ha calculada la similitud entre la
        # fila i-essima de la matriu np_counts i la fila i-essima de matriu np_roll_counts.
        
        # Tractem el cas que el parametre sigui None, seguint les indicacions de l'enunciat de la cel.la seguent
        if similarity_function == None:
            partial_sol = iter_matrix_similarity(np_counts, np_roll_counts)
        # Si el parametre no es None, calculem la similitud amb la funcio passada per parametre
        else:
            partial_sol = similarity_function(np_counts, np_roll_counts)
        
        # Si el nombre d'usuaris es parell i estem a l'ultima iteracio, algunes similituds s'hauran calculat dues
        # vegades, i per tant eliminem una copia de les similituds que s'han calculat dues vegades
        if ((num_users % 2 == 0) and (i == (num_iter - 1))):
            # Eliminem una copia de les similituds que s'han calculat dues vegades
            partial_sol[int(num_users/2):] = 0
        
        # Omplim la diagonal amb les similituds calculades. Es a dir, a la posicio (i,i) de la matriu de similituds
        # hi posem la similitud entre la fila 'i' de la matriu np_counts i una fila 'j' (que dependra de la iteracio en la que
        # ens trobem) de la matriu np_roll_counts
        np.fill_diagonal(sim_matrix, partial_sol)
        
        # Rotem les files de la matriu np_roll_counts. Es a dir, la primera fila de la matriu passa a ser la ultima,
        # la segona fila passa a ser la primera, etc. D'aquesta manera, a la seguent iteracio podrem calcular la similitud
        # entre files diferents.
        np_roll_counts = np.roll(np_roll_counts, -num_cols)
        
        # Rotem els elements de cada fila de la matriu sim_matrix un lloc cap a l'esquerra.
        # Es a dir, el primer element d'una fila passa a ser l'ultim, el segon passa a ser el primer, etc.
        # Amb aixo, esfem fent correr la diagonal de la matriu i per tant a la seguent iteracio, quan omplim la diagonal amb
        # les noves similituds calculades, les estarem col.locant al lloc que realment els hi correspon.
        sim_matrix = np.roll(sim_matrix, -1, axis=1)
    
    #Finalment, rotem els elements de cada fila de la matriu sim_matrix en sentit contrari tants llocs com els hem fet
    # correr abans. Com que en total hem fet 'num_iter' iteracions, desplacem els elements 'num_iter' posicions.
    sim_matrix = np.roll(sim_matrix, num_iter, axis=1)
    
    # Finalment, aprofitem que la matriu es simetrica i li sumem la seva transposada    
    sim_matrix = sim_matrix + sim_matrix.T
    # Posem 1s a tota la diagonal perque 2 vectors iguals tenen similitud 1 i al sumar la matriu amb la transposada la
    # diagonal ha quedat afectada
    np.fill_diagonal(sim_matrix, 1)
    #Retornem la matriu de similituds
    return sim_matrix

Per cridar aquesta funció, el primer paràmetre pot ser:

* Alguna de les funcions que has programat abans (*euclid*, *cosine* o *pearson*) (~@1h 30min treballant directament amb valors de numpy, ~@20h a partir de pandas pur)
* Opcionalment (no és obligatori fer-ho) podeu programar una funció que treballi específicament amb matrius (i no vectors). Si ho feu, cal gestionar-ho quan es rep `None`. No totes les funcions anteriorment anomenades són fàcils (ni intuïtives, ni hi caben en memòria) d'aplicar en forma matricial.  (@5s)

In [12]:
if __name__ == '__main__':
    try:
        with open('similarities.pkl', 'rb') as fp:
            similarities = pickle.load(fp)
    except:
        similarities = similarity_matrix(iter_matrix_similarity, df_reduced)
        with open('similarities.pkl', 'wb') as fp:
            pickle.dump(similarities, fp, pickle.HIGHEST_PROTOCOL)

### Generació de prediccions

Per fer recomanació col·laborativa, necessitem una funció que ens doni un valor de quant bona seria la recomanació. En el nostre cas i amb les nostres dades, volem una funció que ens indiqui quants cops compraria un usuari un producte donat.

* Si esteu fent un recomanador basat en usuaris, la puntuació per a un usuari $u$ i ítem $j$ és

$$pred(u, i) = \hat{r}_{u,i} = \frac{\sum_{p\neq u,r_{p,i}>0} sim(u, p)\cdot r_{p,i}}{\sum_{p\neq u,r_{p,i}>0} sim(u, p)}$$

On $r_{u,i}$ indica el nombre de vegades que l'usuari $u$ ha comprat l'l'ítem $i$.

És a dir, per cada usuari $p$ diferent de $u$ si aquest usuari ha comprat algun cop el producte $i$, la similitud entre $p$ i $u$ multiplicada pel nombre de vegades que l'usuari $p$ ha comprat l'l'ítem $i$ ($r_{p,i}$).

Pondera't per la suma de les similituds.

* Anàlogament, si està basat en ítem, la puntuació per a un usuari $u$ i ítem $j$ és

$$pred(u, i) = \hat{r}_{u,i} = \frac{\sum_{j\neq i,r_{u,j}>0} sim(i, j)\cdot r_{u,j}}{\sum_{j\neq i,r_{u,j}>0} sim(i, j)}$$

On $r_{u,i}$ indica el nombre de vegades que l'usuari $u$ ha comprat l'ítem $j$.

És a dir, per cada ítem $j$ diferent de $i$ si l'usuari al qui recomanem ha comprat l'ítem $j$, la similitud entre $i$ i $j$ multiplicada pel nombre de vegades que l'usuari al qui recomanem $u$ ha comprat l'ítem $j$ ($r_{u,j}$)

Pondera't per la suma de les similituds.

Fixeu-vos que, sigui quin sigui el cas, al final estem fent el producte vectorial entre dos vectors. Concretament, el producte vectorial entre les similituds i les compres. Fes una funció que calculi aquest resultat:

In [13]:
def calc_score(sims, counts):
    """
    * Si estàs implementant basat en usuaris:
        Donades les similituds i el nombre de vegades que l'item a recomanar
        ha estat comprat per cada usuari, retorna la predicció que indica quants
        cops compraria l'usuari un nou ítem.
    
        :param sims: Similituds entre usuaris
        :param counts: Nombre de vegades que l'item a recomanar ha estat comprat per cada usuari
        :return : Predicció (float) que indica quants cops compraria l'usuari un ítem.
        
        
    * Si estàs implementant basat en items:
         Donades les similituds i el nombre de vegades que l'usuari ha comprat
        cada item, retorna la predicció que indica quants cops compraria l'usuari un
        nou ítem.
    
        :param sims: Similituds entre items
        :param counts: Nombre de vegades que l'usuari ha comprat cada item
        :return : Predicció (float) que indica quants cops compraria l'usuari un ítem.
    """
    return np.dot(sims, counts)/np.sum(sims)

In [14]:
# Funcio millorada. Dona mes importancia als usuaris que son realment semblants. El que fem es multiplicar per 1.3
# la similitud dels usuaris que tenen una similitud superior a 0.435
def score(user, item, df, similarities):
    """
    Extreu les similituds i el nombre de vegades que cada usuari ha comprat l'ítem de
    manera normalitzada (resp. nombre de vegades que un usuari ha comprat cada ítem de manera normalitzada)
    i crida a la funció anterior per calcular les prediccions.
    
    :param user: ID de l'usuari per la predicció
    :param item: ID de l'ítem per la predicció
    :param df: Dataframe que conté el nombre de vegades que un usuari 
        ha comprat en un `aisle_id`
    :param similarities: Matriu de similituds
    :return : Retorna un escalar (float) amb la predicció
    
    """
    pos = df.index.get_loc(user)
    sims = similarities[pos].copy()
    sims[pos]= 0
    
    # Multipliquem per 1.3 la similitud dels usuaris que tinguin una similitud superior a 0.435 amb l'usuari 'user' 
    sims[sims > 0.435] = sims[sims > 0.435]*1.3
    
    counts = df[item]
    
    return calc_score(sims, counts)

In [15]:
if __name__ == '__main__':
    print(score(df_reduced.index[0], df_reduced.columns[0], df_reduced, similarities))

0.02393478278219381


Feu una funció que donat un usuari calculi per cada item que no ha comprat la puntuació d'aquest. La funció retorna els $N$ items més ben puntuats.

In [16]:
def recommend_n_items(user_id, df, similarities, N):
    """
    Donat un usuari calcula per cada ítem que no ha comprat la puntuació d'aquest. 
    La funció retorna els $N$ ítems més ben puntuats.
    
    :param user_id: Identificador de l'usuari
    :parma df: Dataframe que conté el nombre de vegades que un usuari 
        ha comprat en un `aisle_id`
    :param similarities: Matriu de similituds
    :param N: Nombre d'ítems que volem que siguin recomanats.
    :return : Llista amb els IDs dels ítems recomanats
    """
    # Seleccionem el nombre de vegades que l'usuari 'user_id' ha comprat cada item
    items = df.loc[user_id]
    # Seleccionem els 'aisle_id' d'aquells items que l'usuari 'user_id' no ha comprat
    items_not_bought = items[items == 0]
    best_items = items_not_bought.index.tolist()
    
    # Ordenem els 'aisle_id' dels items que l'usuari 'user_id' no ha comprat segons la seva puntuacio
    best_items.sort(reverse=True, key=lambda index: score(user_id, index, df, similarities))
    
    #Retornem els N items amb la puntuacio mes alta
    return best_items[:N]

In [17]:
if __name__ == '__main__':
    print(recommend_n_items(df_reduced.index[0], df_reduced, similarities, 10))

[24, 123, 120, 21, 84, 37, 16, 91, 116, 31]


#### Possibles millores 


**0) Utilització completa de les dades:**

Fer servir `df_original` tindrà (possiblement) resultats més fiables, però trigarà molt més que amb la versió reduida `df`. Pots canviar el `FRAC` a valors més alts ($\leq 1$) per utilitzar més dades, però compte perque potser la matriu $M\times M$ no hi cabrà en memòria.

**1) Normalització: Prediccions escalades al domini de l'usuari:**
Els usuaris compren en diferent mesura els productes, un més quantitat, altres menys. Fent servir la següent formula, escalem la predicció  a la mitja de l'usuari:
$$pred(u, i) = \hat{r}_{u,i} = \bar{r_u} + \frac{\sum_{p\neq u,r_{p,i}>0} sim(u, p)\cdot (r_{p,i}-\bar{r_b})}{\sum_{p\neq u,r_{p,i}>0} sim(u, p)}$$
on $\bar{r_u}$ és la mitjana de compres de l'usuari *u*.
    
**2) Valor del nombre d'elements codificats:**
Redueix la similitud entre els usuaris si el nombre de productes és baix o descarta (en entrenament) aquells usuaris amb un petit nombre de productes comprats.

**3) Augment de la similitud:**
Incrementeu el pes als usuaris que són realment similars (~ = 1)

**4) Selecció d'usuaris semblants:**
Només s'utilitza un subconjunt d'usuaris similars, descartant tots aquells usuaris poc similars.


Totes aquestes tècniques es poden aplicar d'igual manera en la recomanació col·laborativa per usuaris o ítems.

# Kaggle

Per als usuaris que tens a continuació, quins productes els hi recomanaries (**fins a un màxim de 5**) que compressin segons el que ja han comprat?

https://www.kaggle.com/t/f5b5030ef8cc4b5dbe985e6033878c75

Si feu modificacions al codi original de cara a millorar els resultats pel Kaggle, feu una còpia d'aquest notebook i treballeu-hi allà.

In [18]:
#En aquesta primera cel.la generem el DataFrame amb el FRAC que ens ha anat milor al Kaggle
if __name__ == '__main__':
    
    df_order_prods = pd.read_csv(join('data', 'order_products__train.csv'))
    df_orders = pd.read_csv(join('data', 'orders.csv'))[['order_id', 'user_id']]
    df_prods = pd.read_csv(join('data', 'products.csv'))[['product_id', 'aisle_id']]
    
    df_merged = merge_information(df_order_prods, df_orders, df_prods)
    
    df_counts = build_counts_table(df_merged)
    
    FRAC = 0.0055
    df_reduced = df_counts.sample(frac=FRAC, random_state=1)

In [19]:
if __name__ == '__main__':
    df_test_products = pd.read_csv(join('data', 'order_products__test.csv'))
    df_test_orders = pd.read_csv(join('data', 'orders__test.csv'))[['order_id', 'user_id']]
    df_test_merged = merge_information(df_test_products, df_test_orders, df_prods)
    
    df_test_counts = build_counts_table(df_test_merged)
    df_all = df_reduced.append(df_test_counts)
    df_all = df_all.fillna(0)

In [20]:
df_all.tail()

aisle_id,1,2,3,4,5,6,7,8,9,10,...,125,126,127,128,129,130,131,132,133,134
user_id,,,,,,,,,,,,,,,,,,,,,
300140,0,0,24,0,0,0,0,0,24,0,...,0,0,0,10,0,10,10,0,0,0
300141,16,0,20,20,16,14,16,20,20,0,...,0,0,0,0,14,0,0,0,0,0
300142,0,0,15,0,15,18,15,18,18,0,...,0,0,0,0,0,0,13,0,0,0
300143,0,0,0,20,0,0,0,0,0,0,...,0,0,0,0,7,20,7,0,0,0
300144,0,0,0,20,12,0,14,0,0,0,...,0,0,0,14,0,0,20,0,0,0


**Tingueu en compte que si voleu fer un canvi a les similarities, heu d'esborrar l'arxiu similarities_test.pkl** (però haureu de recalcular, amb el temps que això suposi).

In [21]:
if __name__ == '__main__':
    try: 
        with open('similarities_test.pkl', 'rb') as fp:
            similarities_test = pickle.load(fp)
    except:
        similarities_test = similarity_matrix(iter_matrix_similarity, df_all)
        with open('similarities_test.pkl', 'wb') as fp:
            pickle.dump(similarities_test, fp, pickle.HIGHEST_PROTOCOL)

In [22]:
if __name__ == '__main__':
    df_submission = pd.DataFrame(columns=['user_id', 'products_list'])

    for user_id in df_test_counts.index:
        user_recos = recommend_n_items(user_id, df_all, similarities_test, 5)

        df_submission = df_submission.append(
            {
                'user_id': user_id,
                'products_list': ' '.join(map(str, user_recos))
            }, 
            ignore_index=True)

    df_submission.to_csv('submission.csv', index=None)

In [23]:
if __name__ == '__main__':
    print(df_submission.head())

  user_id      products_list
0  300000    83 84 91 121 88
1  300001   84 21 120 112 96
2  300002   83 112 121 78 96
3  300003  120 108 91 115 31
4  300004    24 83 21 112 96


Quan tingueu l'arxiu submission.csv generat, podeu pujar-ho a Kaggle com una submission per veure el vostre score.